# Summary of Notebook
    inner join orders on products to reduce null values
    after merging orders with products, the merged file is too large
    searched to understand why.... sleuthing indicated that there may have been duplicate product_ids
    7 duplicate IDs were found: I deleted the ones that were truly duplicates: fiber 4 gm gummy, ranger ipa, black house coffee,  gluten free peanut butter, body wash
    I could change a duplicate product_id to match the index: Revive Zero Vitamin Water now product_id 6799
    There was a duplicate_id for a deodorant and sharp cheddar cheese. After analysis, the deodorant was never purchased. I dropped the row containing the never purchased deodorant
    
    exported file: 30330084
    exported as pickle
    saved as: orders_products_combined.pkl

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import datetime as dt

In [2]:
#import all_orders
df_orders = pd.read_pickle(r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis\02 Data\Prepared Data\all_orders.pkl')
path = r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis'
df_orders = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'all_orders.pkl'))

In [3]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30356421 entries, 0 to 30356420
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int32  
 1   user_id                 int32  
 2   order_number            int16  
 3   day_of_week_ordered     int16  
 4   hour_of_day_ordered     int16  
 5   days_since_prior_order  float64
 6   product_id              int32  
 7   add_to_cart_order       int16  
 8   reordered               int16  
dtypes: float64(1), int16(5), int32(3)
memory usage: 1.1 GB


In [4]:
#import products_cleaned
df_products = pd.read_pickle(r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis\02 Data\Prepared Data\products_cleaned.pkl')
path = r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis'
df_products = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'products_cleaned.pkl'))

In [5]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49677 entries, 0 to 49692
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   product_id     49677 non-null  int32   
 1   product_name   49677 non-null  category
 2   aisle_id       49677 non-null  int16   
 3   department_id  49677 non-null  int16   
 4   price          49677 non-null  float64 
dtypes: category(1), float64(1), int16(2), int32(1)
memory usage: 1.7 MB


In [6]:
#seeing if any data is missing
df_products.isnull().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
price            0
dtype: int64

In [7]:
#seeing if any data is missing
df_orders.isnull().sum()

order_id                  0
user_id                   0
order_number              0
day_of_week_ordered       0
hour_of_day_ordered       0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
dtype: int64

In [8]:
# doing a inner join: I want all orders to have a product information and no nulls
df_combined = df_orders.merge(df_products, on = ('product_id'), indicator = True)

In [9]:
# checking the rows merged on product_id
df_combined['_merge'].value_counts()

both          30329840
left_only            0
right_only           0
Name: _merge, dtype: int64

In [10]:
df_combined.shape

(30329840, 14)

In [11]:
df_orders.shape

(30356421, 9)

#this isn't making sense. The order shape and the combined shape should be the same

In [12]:
#trying to see if there are duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
461,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
6799,6800,Revive Zero Vitamin Water,64,7,6.4
6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
18458,18458,Ranger IPA,27,5,9.2
18459,18458,Ranger IPA,27,5,9.2
26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9
26809,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4


In [13]:
#dropping the duplicate rows
df_products = df_products.drop(labels=[462,18459,26810,35309,35495], axis=0)

In [14]:
#checking to see change in duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
6799,6800,Revive Zero Vitamin Water,64,7,6.4
6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [15]:
# doing a inner join: I want all orders to have a product information and no nulls
df_combined = df_orders.merge(df_products, on = ('product_id'), indicator = True)

In [16]:
df_combined.shape

(30328763, 14)

In [17]:
#trying to see if the products are listed twice or just need a new product_id
found = df_products[df_products['product_name'].str.contains('Revive Zero Vitamin Water')]
print(found.count())

product_id       1
product_name     1
aisle_id         1
department_id    1
price            1
dtype: int64


In [18]:
#trying to see if the products are listed twice or just need a new product_id
found = df_products[df_products['product_name'].str.contains('Sprouted Quinoa Flakes Baby Cereal')]
print(found.count())

product_id       1
product_name     1
aisle_id         1
department_id    1
price            1
dtype: int64


In [19]:
#trying to see if the products are listed twice or just need a new product_id
found = df_products[df_products['product_name'].str.contains('Clinical Advanced Solid Ultimate Fresh')]
print(found.count())

product_id       1
product_name     1
aisle_id         1
department_id    1
price            1
dtype: int64


In [20]:
#trying to see if the products are listed twice or just need a new product_id
found = df_products[df_products['product_name'].str.contains('Cheese Shredded Sharp Cheddar Reduced Fat 2%')]
print(found.count())

product_id       1
product_name     1
aisle_id         1
department_id    1
price            1
dtype: int64


#each item is only in the product_name column once so the name is not duplicated. Based on the pattern of index number and product_id, I am going to change the product_id to match the index number

In [21]:
#checking to see change in duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
6799,6800,Revive Zero Vitamin Water,64,7,6.4
6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [22]:
#attempting to change the product_id
df_products.at[6799, 'product_id']

6800

In [23]:
#attempting to change the product_id
df_products.at[6799, 'product_id']=6799

In [24]:
#checking to see change in duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [25]:
#attempting to change the product_id
df_products.at[26521, 'product_id']=26521

In [26]:
#checking to see change in duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
26521,26521,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9
26522,26521,Organic Grass-Fed Classic Mild Cheddar Macaron...,4,9,9.0


In [27]:
# doing a inner join: I want all orders to have a product information and no nulls
df_combined = df_orders.merge(df_products, on = ('product_id'), indicator = True)

In [28]:
df_combined.shape

(30331681, 14)

In [29]:
df_orders.shape

(30356421, 9)

#this isn't working If I were on a team, I'd just ask for the product_id

In [30]:
#attempting to change the product_id
df_products.at[26521, 'product_id']=26520

In [31]:
#checking to see change in duplicated product_id
df_products[df_products.duplicated(['product_id'], keep=False)]

,product_id,product_name,aisle_id,department_id,price
26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [32]:
# doing a inner join: I want all orders to have a product information and no nulls
df_combined = df_orders.merge(df_products, on = ('product_id'), indicator = True)

In [33]:
df_combined.shape

(30330612, 14)

In [34]:
#counting how many times the product_name clinincal advances solid ultimate happens
(df_combined['product_name']=='Clinical Advanced Solit Ultimate Fresh').sum()

0

In [35]:
#counting how many times the product_name cheese shredded sharp cheddar reduced fat ultimate happens
(df_combined['product_name']=='Cheese Shredded Sharp Cheddar Reduced Fat 2%').sum()

528

#based on this analysis, I can drop the row named as index 26520

In [36]:
#dropping the duplicate row
df_products = df_products.drop(labels=[26520], axis=0)

In [37]:
# doing a inner join: I want all orders to have a product information and no nulls
df_combined = df_orders.merge(df_products, on = ('product_id'), indicator = True)

In [38]:
df_combined.shape

(30330084, 14)

In [39]:
df_orders.shape

(30356421, 9)

In [40]:
#seeing if any data is missing
df_combined.isnull().sum()

order_id                  0
user_id                   0
order_number              0
day_of_week_ordered       0
hour_of_day_ordered       0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
product_name              0
aisle_id                  0
department_id             0
price                     0
_merge                    0
dtype: int64

In [41]:
#exporting merged file as a pickle
df_combined.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))